# Verifying false-positive rates by simulation

These simulations are similar to those found in the `FPR-simulation.ipynb` notebook, but they use chi-squared random variables instead of standard normals. 

In [1]:
from mne.parallel import parallel_func
from niseq.max_test import (
    sequential_permutation_t_test_1samp, 
    sequential_permutation_test_indep
)
import numpy as np

N_SIMULATIONS = 10000

On each simulation below, we'll generate null data and pretend we look at it five times throughout the intended course of data collection, and we'll compare the false positive rates attained when we reject the null hypothesis whenever $p \leq 0.05$ and $p \leq \alpha_\text{adjusted}$ at at least one look time.

In [2]:
def one_simulation(seed, 
                n_tests = 1,
                tail = 0, 
                indep = False,
                look_times = np.linspace(100, 500, 5).astype(int)):
    
    rng = np.random.default_rng(seed)
    x = rng.chisquare(df = 5, size = (look_times[-1], n_tests))
    
    ## run sequential test
    if indep: # independent samples test
        x = rng.chisquare(df = 5, size = (look_times[-1], n_tests))
        conds = rng.choice([0, 1], look_times[-1])
        _, p, adj_alpha, _ = sequential_permutation_test_indep(
            x, conds, look_times, n_max = look_times[-1], 
            tail = tail,
            seed = seed,
            verbose = False
        ) 
    else: # paired-sample test
        x0 = rng.chisquare(df = 5, size = (look_times[-1], n_tests))
        x1 = rng.chisquare(df = 5, size = (look_times[-1], n_tests))
        x = x1 - x0
        _, p, adj_alpha, _ = sequential_permutation_t_test_1samp(
            x, look_times, n_max = look_times[-1], 
            tail = tail,
            seed = seed,
            verbose = False
        ) 
        
    # reject if p-val crosses sig threshold at any look time
    return np.array([np.any(p < .05), np.any(p < adj_alpha)]) 


def run_simulations(n_simulations, n_tests = 1, tail = 0, indep = False, n_jobs = -1):
    parallel, p_func, _ = parallel_func(one_simulation, n_jobs)
    out = parallel(p_func(seed, n_tests, tail, indep) for seed in range(n_simulations))
    rejections = np.stack(out)
    fpr = rejections.mean(0)
    print('False positive rate without correction: ' + str(fpr[0]))
    print('False positive rate *with* correction: ' + str(fpr[1]))

## One Sample Test

In [3]:
# t-max correction for 100 tests
run_simulations(N_SIMULATIONS, n_tests = 100, indep = False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   40.9s
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 282 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 444 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 642 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 1146 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 1452 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 1794 tasks      | elapsed: 16.5min
[Parallel(n_jobs=-1)]: Done 2172 tasks      | elapsed: 20.0min
[Parallel(n_jobs=-1)]: Done 2586 tasks      | elapsed: 23.8min
[Parallel(n_jobs=-1)]: Done 3036 tasks      | elapsed: 27.9min
[Parallel(n_jobs=-1)]: Done 3522 tasks      | elapsed: 32.3min
[Parallel(n_jobs=-1)]: Done 4044 tasks      | ela

False positive rate without correction: 0.1794
False positive rate *with* correction: 0.0452


[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 90.9min finished


## Independent Sample Test

In [4]:
# t-max correction
run_simulations(N_SIMULATIONS, n_tests = 100, indep = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   54.9s
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 282 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 444 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 642 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 1146 tasks      | elapsed: 15.7min
[Parallel(n_jobs=-1)]: Done 1452 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done 1794 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done 2172 tasks      | elapsed: 29.6min
[Parallel(n_jobs=-1)]: Done 2586 tasks      | elapsed: 35.3min
[Parallel(n_jobs=-1)]: Done 3036 tasks      | elapsed: 41.4min
[Parallel(n_jobs=-1)]: Done 3522 tasks      | elapsed: 47.9min
[Parallel(n_jobs=-1)]: Done 4044 tasks      | ela

False positive rate without correction: 0.1812
False positive rate *with* correction: 0.0486


[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 137.1min finished
